In [1]:
import networkx as nx
import pandas as pd
from pyvis.network import Network
import ibm_db
import ibm_db_dbi

In [2]:
#conectando ao BigSql
#COLOCAR CREDENCIAIS
db = ibm_db.connect("DATABASE=BIGSQL;HOSTNAME=bigsql.pro.intra.rs.gov.br;PORT=32051;PROTOCOL=TCPIP;UID=XXXXX;PWD=XXXXX;", "", "")
conn = ibm_db_dbi.Connection(db)   

In [3]:
# buscando os dados das arestas / quem se relaciona com quem
query = "select * from PROCERGS_MILENA_VILLAR.RELACIONAMENTO2;"
df = pd.read_sql_query(query, conn)
# alterando os nomes das colunas para letras minúsculas
df.rename(str.lower, axis='columns', inplace = True)
df.head()

,src,tgt,rel,weight,edge_key
0,3,4,irmã,3,F
1,5,4,co-autores,4,C
2,5,4,co-autores,4,C
3,6,7,co-autores,2,C
4,7,6,co-autores,2,C


In [4]:
#informando quais colunas da tabela serão o source, target e demais atributos;
#create_using específica qual é o tipo do grafo
# edge_key diferencia / categoriza o tipo de relação
M = nx.from_pandas_edgelist(df, source = 'src', target = 'tgt', edge_key='edge_key', edge_attr=['rel','weight'], create_using=nx.MultiDiGraph())

In [5]:
# verificando se o grafo é ponderado
nx.is_weighted(M)

True

In [6]:
# verificando se importou os edges corretamente
sorted(M.edges(data = True, keys = True))[:5]

[(1, 2, 'C', {'rel': 'co-autores', 'weight': 4}),
 (1, 3, 'F', {'rel': 'mãe', 'weight': 5}),
 (1, 4, 'F', {'rel': 'mãe', 'weight': 5}),
 (1, 10, 'F', {'rel': 'irmão', 'weight': 3}),
 (1, 16, 'F', {'rel': 'irmã', 'weight': 3})]

In [7]:
# plotar o grafo
nt = Network("400px", "900px", notebook = True, directed = False)
nt.from_nx(M)
nt.show("m.html")

In [7]:
#buscando os labels dos nós(vértices)
query = "select * from PROCERGS_MILENA_VILLAR.ATRIBUTOS;"
df_node = pd.read_sql_query(query, conn)

In [8]:
df_node.head()

,ID,NOME,SZ,GP,LAT,LONGT,P_ARMA,TIPO_ARMA,TIPO_CRIME,NUM_VITIMAS
0,1,Vivianne,15,1,-29.881725,-51.164042,SIM,2,ASSALTO,5
1,3,Jade,5,1,-29.881725,-51.164042,NÃO,0,NÃO,0
2,5,Eduardo,5,2,-29.894814,-51.159835,SIM,2,ASSALTO,3
3,6,Ricardo,5,3,-29.916482,-51.146204,SIM,1,TRAF DROG,0
4,7,Inês,15,3,-29.916482,-51.146204,NÃO,0,NÃO,0


In [9]:
#transformando a tabela em um dicionário para atribuir labels aos IDs dos nós
nodelabel = df_node.set_index("ID").T.to_dict('records')[0]
# atribuindo o atributo label
nx.set_node_attributes(M, nodelabel, 'title')
#M.nodes(data=True)

In [10]:
#buscando os tamanhos dos nós(vértices)
query = "select ID,SZ from PROCERGS_MILENA_VILLAR.ATRIBUTOS;"
a = pd.read_sql_query(query, conn)

In [11]:
#transformando a tabela em um dicionário para atribuir os tamanhos aos IDs dos nós
nodeatrib = a.set_index("ID").T.to_dict('records')[0]
# atribuindo o atributo label
nx.set_node_attributes(M, nodeatrib, 'size')
#M.nodes(data=True)

In [12]:
#buscando os grupos dos nós(vértices)
query = "select ID,GP from PROCERGS_MILENA_VILLAR.ATRIBUTOS;"
g = pd.read_sql_query(query, conn)

In [13]:
#transformando a tabela em um dicionário para atribuir os grupos aos IDs dos nós
nodeatrib = g.set_index("ID").T.to_dict('records')[0]
# atribuindo o atributo label
nx.set_node_attributes(M, nodeatrib, 'group')
#M.nodes(data=True)

In [14]:
#buscando o atributo coordenadas dos nós(vértices)
query = "select * from PROCERGS_MILENA_VILLAR.COORDENADAS;"
at_node = pd.read_sql_query(query, conn)
#at_node.head()

In [15]:
#transformando a tabela em um dicionário
c = at_node.set_index("ID").T.to_dict('list')
#adicionando o atributo coordenada
nx.set_node_attributes(M, c, 'pos')
#M.nodes(data=True)

In [25]:
#buscando TIPO DE CRIME, NUM. VÍTIMAS dos nós(vértices)
query = "select ID,P_ARMA,TIPO_ARMA,TIPO_CRIME,NUM_VITIMAS from PROCERGS_MILENA_VILLAR.ATRIBUTOS;"
g = pd.read_sql_query(query, conn)

In [26]:
#transformando a tabela em um dicionário para atribuir aos IDs dos nós
nodeatrib = g.set_index("ID").T.to_dict()
# atribuindo o atributo 
nx.set_node_attributes(M, nodeatrib)
#M.nodes(data=True)

In [27]:
# plotar o grafo
nt = Network("400px", "900px", notebook = True)
nt.toggle_hide_edges_on_drag(False) # esconde as arestas quando clica com o mouse
#nt.barnes_hut() #tipo de layout
# nt.show_buttons(filter_=['physics'])
nt.from_nx(M)
nt.show("mg.html")

## Criando Subgrafos para categorizar as arestas

In [28]:
# filtrando somente as arestas com chave igual 'C' (relacionamento criminoso)
co_autores = [(u, v, k) for u, v, k in M.edges if k == 'C']
print(sorted(co_autores))

[(1, 2, 'C'), (2, 1, 'C'), (3, 12, 'C'), (3, 13, 'C'), (4, 8, 'C'), (4, 9, 'C'), (4, 12, 'C'), (4, 13, 'C'), (5, 4, 'C'), (6, 7, 'C'), (7, 6, 'C'), (8, 4, 'C'), (9, 4, 'C'), (10, 11, 'C'), (11, 10, 'C'), (12, 3, 'C'), (12, 4, 'C'), (13, 3, 'C'), (13, 4, 'C'), (14, 15, 'C'), (15, 14, 'C'), (17, 18, 'C'), (18, 17, 'C'), (19, 4, 'C'), (19, 20, 'C'), (20, 4, 'C')]


In [29]:
# criando um subgrafo somente com os relacionamentos entre co-autores
C = nx.MultiDiGraph.edge_subgraph(M,co_autores)
#Verificando se criou o subgrafo
list(C.edges(data = True, keys = True))[:5]

[(3, 13, 'C', {'rel': 'co-autores', 'weight': 1}),
 (3, 12, 'C', {'rel': 'co-autores', 'weight': 1}),
 (4, 8, 'C', {'rel': 'co-autores', 'weight': 3}),
 (4, 9, 'C', {'rel': 'co-autores', 'weight': 3}),
 (4, 13, 'C', {'rel': 'co-autores', 'weight': 1})]

In [19]:
nt = Network("400px", "900px", notebook = True)
nt.from_nx(C)
nt.show("c.html")

In [20]:
#número total de nós
C.number_of_nodes()

19

In [21]:
# número total de arestas
C.number_of_edges()

26

In [30]:
# consultando atributo por edge e node
# buscando nós que possuem o relacionamento 'co-autores' e estão no grupo '2'
q = set(n for u,v,d in C.edges(data=True)
               if d['rel']=='co-autores'
               for n in (u, v)
               if C.nodes[n]['group'] == 2)
print(q)

{4, 5}


In [32]:
#Consultando atributo do node
grupo = (n for n in C if int(C.nodes[n]['NUM_VITIMAS']) >= 6)
print(list(grupo))

[10]


In [33]:
# Adicionando um atributo em uma determinada aresta do subgrafo C
C.edges[3,13,'C']['cidade_atuacao'] = 'CANOAS'


In [34]:
# verificando se realmente acrescentou atributo ao subgrafo C
list(C.edges(data = True, keys = True))[:5]

[(3, 13, 'C', {'rel': 'co-autores', 'weight': 1, 'cidade_atuacao': 'CANOAS'}),
 (3, 12, 'C', {'rel': 'co-autores', 'weight': 1}),
 (4, 8, 'C', {'rel': 'co-autores', 'weight': 3}),
 (4, 9, 'C', {'rel': 'co-autores', 'weight': 3}),
 (4, 13, 'C', {'rel': 'co-autores', 'weight': 1})]

In [35]:
# verificando se alterou também no grafo principal M
list(M.edges(data = True, keys = True))[:5]

[(3, 4, 'F', {'rel': 'irmã', 'weight': 3}),
 (3, 13, 'C', {'rel': 'co-autores', 'weight': 1, 'cidade_atuacao': 'CANOAS'}),
 (3, 12, 'C', {'rel': 'co-autores', 'weight': 1}),
 (4, 3, 'F', {'rel': 'irmã', 'weight': 3}),
 (4, 8, 'C', {'rel': 'co-autores', 'weight': 3})]

### Descobrindo relação entre os nós

In [37]:
# Verificando se existe uma aresta entre dois nós
C.has_edge(3,9)

False

In [39]:
# Descobre o caminho de custo mínimo (ccm) do vértice 3 até 9 // o peso das arestas é considerado
try:
    ccm = nx.dijkstra_path(C,source = 3, target = 9, weight = 'weight') 
    tam = nx.dijkstra_path_length(C,source = 3, target = 9, weight = 'weight')
    print('Caminho de custo mínimo: ', ccm)
    print('Tamanho do caminho: ', tam) # soma dos pesos das arestas
except nx.NetworkXNoPath:
    print("Caminho não encontrado") 

Caminho de custo mínimo:  [3, 13, 4, 9]
Tamanho do caminho:  5


In [46]:
# listando todo os caminhos mais curtos // // o peso das arestas é considerado
try:
    for path in nx.all_shortest_paths(C, source= 3, target = 9, weight = 'weight', method= 'dijkstra'):
        print(path)
except nx.NetworkXNoPath:
    print("Caminho não encontrado") 

[3, 13, 4, 9]
[3, 12, 4, 9]


In [41]:
# Informa o tamanho e o caminho mais curto
tam, cam = nx.single_source_dijkstra(C, source= 3, target= 9, cutoff= None, weight='weight')
tam, cam

(5, [3, 13, 4, 9])